# Simulación de personalización de noticias usando Bandidos Contextuales

Vamos a simular un escenario en que debemos personalizar el contenido de un sitio, usando Bandidos Contextuales (CB). El objetivo es maximizar el Click Through Rate (CTR)

Recordemos CB, cada datapoint tiene 4 componentes: 

- Contexto
- Acción
- Probabilidad de tomar la acción
- Recompensa/costo recibida


En el simulador necesitamos generar contexto, obtener una acción para cada contexto y también simular la recompensa.

En el simulador, el objetivo es maximizar la recompensa (CTR) o minizar la pérdida (-CTR).


- Tenemos dos visitantes al sitio, 'Tom' y 'Anna'
- Cada un do ellos puede visitar el sitio a la mañana o a la tarde

El **contexto** es entonces (usuario, tiempo_del_dia)

Tenemos la opción de recomendar una variedad de artículos. Las acciones son entonces las diferentes opciones: "politica", "deportes", "musica", "comida", "finanzas", "salud" y "queso"

La **recompensa** es si cliquean o no en el artículo: 'click' o 'no click'

Importamos algunas cosas:

In [ ]:
from vowpalwabbit import pyvw
import random
import matplotlib.pyplot as plt
import numpy as np

## Simular la recompensa

En el mundo real vamos a tener que aprender las preferencias de Tom y Anna por los artículos observando sus interacciones. Como esta es una simulación vamos a definir un perfil de preferencias de Tom y Anna. La recompensa que proveemos a un agente (que aprende) va a respetar este perfil de preferencia. Nuestra esperanza es que el agente tome cada vez mejores decisiones a medida que observa más ejemplo y maximice así la recompensa.

Tambié vamos a modificar la función de recompensa de diferentes maneras (así no es estacionario el problema) y ver si el agente lo aprende. Vamos a compara el CTR con aprendizaje y sin aprendizaje.

Vowpal Wabbit optimiza la función de costo, **que es una recompensa negativa**. 

In [ ]:
# Como VW trata de minizar costo/pérdida, usamos -recompensa
USER_LIKED_ARTICLE = -1.0
USER_DISLIKED_ARTICLE = 0.0

La función de recompensa de abajo especifíca que a Tom le gusta leer de política a la mañana y música a la tarde mientras que Anna prefiere leer de deportes a la mañana y de política a la tarde. Si el agente recomienda un artículo que se aliñea con la función de recompensa entonces recibimos una recompensa positiva. En este caso simulado, un click.

In [ ]:
def get_cost(context,action):
    if context['user'] == "Tom":
        if context['time_of_day'] == "morning" and action == 'politics':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'music':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE
    elif context['user'] == "Anna":
        if context['time_of_day'] == "morning" and action == 'sports':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'politics':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE


## El formato de Vowpal Wabbit

Hay algunas cosas que hay que hacerle al input para que VW lo entienda. La siguiente función que se ocupa de convertir desde el contexto, lista de artículos y recompensa a un formato que VW entienda.

In [ ]:
# Esta función modifica la tupla (contexto, acción, costo, probabilidad) a un formato amigable a VW
def to_vw_example_format(context, actions, cb_label = None):
    if cb_label is not None:
        chosen_action, cost, prob = cb_label
    example_string = ""
    example_string += "shared |User user={} time_of_day={}\n".format(context["user"], context["time_of_day"])
    for action in actions:
        if cb_label is not None and action == chosen_action:
            example_string += "0:{}:{} ".format(cost, prob)
        example_string += "|Action article={} \n".format(action)
    #remover el último pasaje de línea (\n)
    return example_string[:-1]

Para entener qué está pasando veamos un ejemplo. Aqui, es de mañana y el usuario es Tom. Hay cuatro artículos posibles. Así que en el formato de VW hay una línea que empiezaz con shared (o sea el contexto que fue 'compartido'), segudio de cuatro líneas correspondientes a cada artículo.

In [ ]:
context = {"user":"Tom","time_of_day":"morning"}
actions = ["politics", "sports", "music", "food"]

print(to_vw_example_format(context,actions))

## Obteniendo una decisión

Cuando llamos a VW obtenemos una distribución de probabilidad sobre acciones como salida. En cada índice está la probabilidad de tomar la acción correspondiente a ese índice. Para tomar una decisión 'sampleamos' acciones con esas probabilidades.

In [ ]:
def sample_custom_pmf(pmf):
    total = sum(pmf)
    scale = 1/total
    pmf = [x * scale for x in pmf]
    index = np.random.choice(range(len(pmf)), p=pmf)
    return index, pmf[index]

We have all of the information we need to choose an action for a specific user and context. To use VW to achieve this, we will do the following:

1. We convert our context and actions into the text format we need
2. We pass this example to vw and get the pmf out
3. Now, we sample this pmf to get what article we will end up showing
4. Finally we return the article chosen, and the probability of choosing it (we are going to need the probability when we learn form this example)

In [ ]:
def get_action(vw, context, actions):
    vw_text_example = to_vw_example_format(context,actions)
    pmf = vw.predict(vw_text_example)
    chosen_action_index, prob = sample_custom_pmf(pmf)
    return actions[chosen_action_index], prob

## Setup de la simulación

Ahora simulemos el mundo de Tom y Anna. El escenario es que van a un sitio y les muestran un artículo. Vamos a elegir entre Tom y Anna aleatoriamente de manera uniforme y también el tiempo de visita de manera uniforme.

In [ ]:
users = ['Tom', 'Anna']
times_of_day = ['morning', 'afternoon']
actions = ["politics", "sports", "music", "food", "finance", "health", "camping"]

def choose_user(users):
    return random.choice(users)

def choose_time_of_day(times_of_day):
    return random.choice(times_of_day)

Instanciamos un agente de CB in VW y simulamos las visitas de Tom y Anna durante `num_iterations` veces. En cada visita:

We will instantiate a CB learner in VW and then simulate Tom and Anna's website visits `num_iterations` number of times. In each visit, we:

1. Decidimos entre Tom y Anna
2. Decidimos el tiempo del día
3. Pasamos un contexto (usuario, tiempo del dia) al agente para obtener una acción (una recomendación de artículo y la probabilidad)
4. Recibimos una recompensa (si el usuario cliqueó). Recordar que el costo es la recompensa negativa.
5. Formatamos el contextio, acción, probabilidad y recompensa en e formato VW.
6. Aprendemos del ejemplo.

Eso es igual para todas las simulaciones más abajo y se usa la función `run_simulation`.

In [ ]:
def run_simulation(vw, num_iterations, users, times_of_day, actions, cost_function, do_learn = True):
    cost_sum = 0.
    ctr = []

    for i in range(1, num_iterations+1):
        # 1. En cada simulación elegir el usuario
        user = choose_user(users)
        # 2. Elegir el tiempo del día para el usuario
        time_of_day = choose_time_of_day(times_of_day)

        # 3. pasar el contexto a VW para obtener la acción
        context = {'user': user, 'time_of_day': time_of_day}
        action, prob = get_action(vw, context, actions)

        # 4. obtener el costo de la acción tomada
        cost = cost_function(context, action)
        cost_sum += cost

        if do_learn:
            # 5. informar a VW de lo que ocurrió y para poder aprender
            vw_format = vw.parse(to_vw_example_format(context, actions, (action, cost, prob)),pyvw.vw.lContextualBandit)
            # 6. aprender
            vw.learn(vw_format)

        # hacemos -costo para obtener CTR
        ctr.append(-1*cost_sum/i)

    return ctr

Queremos poder visualizar lo que ocuerre, así qye vamos a plotear el CTR en cada iteración de la simulación. Si VW toma acciones que obtienen recompensas el CTR va a ser más alto. La función más abajo plotea.

In [ ]:
def plot_ctr(num_iterations, ctr):
    plt.plot(range(1,num_iterations+1), ctr)
    plt.xlabel('num_iterations', fontsize=14)
    plt.ylabel('ctr', fontsize=14)
    plt.ylim([0,1])

## Escenario 1

Vamos a usar la función de recompensa `get_cost` y asumir que Tom y Anna no cambian sus preferencias a lo largo del tiempo y ver qué pasa con el CTR mientras se aprende. También comparamos con el baseline cuando no se aprende.


### Con aprendizaje:

In [ ]:
# instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations = 1000
ctr = run_simulation(vw, num_iterations, users, times_of_day, actions, get_cost)

plot_ctr(num_iterations, ctr)

### nota: interacciones
Uno de los argumentos que pasamos a VW es `-q UA`. Esto le dice a VW que cree features adicionales que son las features del usuario y acción multiplicadas. Esto permite aprender de interacciones entre tiempos del día/usuario y acciones. Si no el aprendizaje con VW no funciona. Se ve abajo:

In [ ]:
# Instanciar VW per sin -q
vw = pyvw.vw("--cb_explore_adf --quiet --epsilon 0.2")

num_iterations = 1000
ctr = run_simulation(vw, num_iterations, users, times_of_day, actions, get_cost)

plot_ctr(num_iterations, ctr)

### Sin aprendizaje
Hagamos lo mismo de nuevo (pero con `-q`, y esta vez sin aprender, el CTR no mejora y oscilar alrededor de 0.2)

In [ ]:
# Instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations = 1000
ctr = run_simulation(vw, num_iterations, users, times_of_day, actions, get_cost, do_learn=False)

plot_ctr(num_iterations, ctr)

## Escenario 2

En el mundo real las preferencias de la gente cambian con el tiempo. Ahora simulamos eso incorporando dos funciones de costo. Y cambiamos una por la otra en la mitad del experimento.

### Tom

| | `get_cost` | `get_cost_new1` |
|:---|:---:|:---:|
| **Mañana** | Politics | Politics |
| **Tarde** | Music | Sports |

### Anna

| | `get_cost` | `get_cost_new1`  |
|:---|:---:|:---:|
| **Mañana** | Sports | Sports |
| **Tarde** | Politics | Sports |


In [ ]:
def get_cost_new1(context,action):
    if context['user'] == "Tom":
        if context['time_of_day'] == "morning" and action == 'politics':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'sports':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE
    elif context['user'] == "Anna":
        if context['time_of_day'] == "morning" and action == 'sports':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'sports':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE

Para hacerlo simple mostramos el efecto de la función de costo que estamos cambiando modificamos la función `run_simulation`. Permite aceptar una lista de funciones de costo y va a cambiar de una a otra por vez.

In [ ]:
def run_simulation_multiple_cost_functions(vw, num_iterations, users, times_of_day, actions, cost_functions, do_learn = True):
    cost_sum = 0.
    ctr = []

    start_counter = 1
    end_counter = start_counter + num_iterations
    for cost_function in cost_functions:
        for i in range(start_counter, end_counter):
            # 1.en cada simulacion elegir el usuario
            user = choose_user(users)
            # 2. elegir el tiempo del dia para cada usuario
            time_of_day = choose_time_of_day(times_of_day)

            # construir contexto
            context = {'user': user, 'time_of_day': time_of_day}

            # 3. obtener accion
            action, prob = get_action(vw, context, actions)

            # 4. obtener costo de la accion tomada
            cost = cost_function(context, action)
            cost_sum += cost

            if do_learn:
                # 5. informar a VW de lo que pasó
                vw_format = vw.parse(to_vw_example_format(context, actions, (action, cost, prob)),pyvw.vw.lContextualBandit)
                # 6. aprender
                vw.learn(vw_format)

            # hacemos -costo para obtener CTR
            ctr.append(-1*cost_sum/i)
        start_counter = end_counter
        end_counter = start_counter + num_iterations
    return ctr

### Con aprendizaje

Ahora prendemos la segunda función de recompensa luego de algunos ejemplos. Esperamos ver cómo el agente aprende este cambio de situación.

In [ ]:
# usar la primera función de recompensa inicialmente y luego cambiar a la segunda

# instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations_per_cost_func = 1000
cost_functions = [get_cost, get_cost_new1]
total_iterations = num_iterations_per_cost_func * len(cost_functions)

ctr = run_simulation_multiple_cost_functions(vw, num_iterations_per_cost_func, users, times_of_day, actions, cost_functions)

plot_ctr(total_iterations, ctr)


**Nota:** El pico inicial de CTR depende de las recompensas iniciales en los primeros ejemplos. Esto es un aleatorio en cada corrida.

### Sin aprendizaje

In [ ]:
# no aprender
# usar la primera función de recompensa inicialmente y luego cambiar a la segunda

# instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations_per_cost_func = 1000
cost_functions = [get_cost, get_cost_new1]
total_iterations = num_iterations_per_cost_func * len(cost_functions)

ctr = run_simulation_multiple_cost_functions(vw, num_iterations_per_cost_func, users, times_of_day, actions, cost_functions, do_learn=False)
plot_ctr(total_iterations, ctr)

## Escenario 3
En este escenario empezamos a recompensar acciones que nunca vieron una recompensa previamente cuando cambiamos la función de costo.

### Tom

| | `get_cost` | `get_cost_new2` |
|:---|:---:|:---:|
| **Mañana** | Politics |  Politics|
| **Tarde** | Music |   Food |

### Anna

| | `get_cost` | `get_cost_new2` |
|:---|:---:|:---:|
| **Mañana** | Sports | Food|
| **Tarde** | Politics |  Food |


In [ ]:
def get_cost_new2(context,action):
    if context['user'] == "Tom":
        if context['time_of_day'] == "morning" and action == 'politics':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'food':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE
    elif context['user'] == "Anna":
        if context['time_of_day'] == "morning" and action == 'food':
            return USER_LIKED_ARTICLE
        elif context['time_of_day'] == "afternoon" and action == 'food':
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE


### Con aprendizaje
Ahora la función  de recompensa funciona con espacio de acción **diferente** después de un tiempo. Deberíamos ver al agente aprender esto.


In [ ]:
# usar la primera función de recompensa inicialmente y luego cambiar a la tercera

# instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations_per_cost_func = 1000
cost_functions = [get_cost, get_cost_new2]
total_iterations = num_iterations_per_cost_func * len(cost_functions)

ctr = run_simulation_multiple_cost_functions(vw, num_iterations_per_cost_func, users, times_of_day, actions, cost_functions)

plot_ctr(total_iterations, ctr)

### Sin aprendizaje

In [ ]:
# no aprender
# usar la primera función de recompensa inicialmente y luego cambiar a la tercera

# instanciar agente en VW
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations_per_cost_func = 1000
cost_functions = [get_cost, get_cost_new2]
total_iterations = num_iterations_per_cost_func * len(cost_functions)

ctr = run_simulation_multiple_cost_functions(vw, num_iterations_per_cost_func, users, times_of_day, actions, cost_functions, do_learn=False)

plot_ctr(total_iterations, ctr)

## Resumen

Este tutorial trata de mostrar un escenario real en una situación muy simplificada donde se pueden usar bandidos contextuales. Vimos que se puede tomar un contexto y una serie de acciones y cómo se puede aprender cuáles son óptimas para cada contexto. Vimos que el agente puede aprender rápidamente a los cambios del munco. 

Este tutorial es muy simplificado, pero VW soporta features de altas dimensiones y diferentes algoritmos de exploración y estrategias evaluación de políticas.